Import


In [2]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

création de la dataframe

In [3]:
df = pd.read_csv(r"After_Sort.csv", parse_dates=["BaseDateTime"])
df.sort_values("BaseDateTime", inplace=True)

print informations

In [4]:
print(df.head())
print(df.info())
print(df.describe())
print(df["VesselType"].value_counts())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO CallSign  VesselType  Status  \
117352    206.0       MERCURY SKY     IMO9796949   9V5148          70       0   
117395    294.0  PROVINCETOWN III     IMO9329394  WDB8185          60       0   
117339    184.0         COBIA LNG     IMO9869306  9HA5192          84       0   
117360      NaN   BELLE CHASSE II            NaN  WDD7069          60       0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  WDK3453          60       0   

        Length  Width  Draft  Cargo TransceiverClass  
1

ajout des colones lat et long +5, +10, +15 à la dataframe

In [ ]:
nouvelles_colonnes = ['LONG_5', 'LAT_5', 'LONG_10', 'LAT_10', 'LONG_15', 'LAT_15', 'Delta_temps']

# Ajout des colonnes avec des valeurs vides (NaN)
for col in nouvelles_colonnes:
    df[col] = None  # ou pd.NA, ou np.nan si vous utilisez numpy


print(df.head())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO  ... Width  Draft  Cargo  \
117352    206.0       MERCURY SKY     IMO9796949  ...   NaN    NaN   70.0   
117395    294.0  PROVINCETOWN III     IMO9329394  ...   9.0    1.9   60.0   
117339    184.0         COBIA LNG     IMO9869306  ...  46.0    9.4   84.0   
117360      NaN   BELLE CHASSE II            NaN  ...  16.0    NaN   60.0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  ...  16.0    3.3   60.0   

        TransceiverClass  LONG_5  LAT_5  LONG_10 LAT_10 LONG_15 LAT_15  
117352 

In [6]:

#Séparation en bases d'apprentissage, test et validation par MMSI

# Récupération des MMSI uniques et suppr valeurs manquantes
mmsi_uniques = df['MMSI'].dropna().unique()

# Mélange aléatoire
np.random.seed(42)
np.random.shuffle(mmsi_uniques)

# Affectation fixe : 97 pour apprentissage, 30 pour test, 22 pour validation
n_train = 97
n_test = 30
n_val = 22

mmsi_train = mmsi_uniques[:n_train]
mmsi_test = mmsi_uniques[n_train:n_train + n_test]
mmsi_val = mmsi_uniques[n_train + n_test:n_train + n_test + n_val]

# Création des trois sous-ensembles
df_train = df[df['MMSI'].isin(mmsi_train)]
df_test = df[df['MMSI'].isin(mmsi_test)]
df_val = df[df['MMSI'].isin(mmsi_val)]

# Affichage pour vérification
print("Nombre de MMSI uniques:")
print("Train:", df_train['MMSI'].nunique())
print("Test :", df_test['MMSI'].nunique())
print("Val  :", df_val['MMSI'].nunique())




Nombre de MMSI uniques:
Train: 97
Test : 30
Val  : 22


création de la dataframe

In [19]:
print(df_train.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_test.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_val.head(5))

             id       MMSI        BaseDateTime       LAT       LON   SOG  \
117354  9030354  209513000 2023-05-25 00:00:04  26.34658 -78.94057  31.9   
117425  9035334  209513000 2023-05-25 00:01:06  26.33986 -78.94755  31.9   
117521  9042882  209513000 2023-05-25 00:02:16  26.33224 -78.95551  32.2   
118718  9138497  209513000 2023-05-25 00:11:58  26.26969 -79.02179  31.6   
118261  9101604  209513000 2023-05-25 00:14:42  26.25213 -79.04084  32.3   

          COG  Heading   VesselName         IMO  ... Width  Draft  Cargo  \
117354  222.8    224.0  OCEAN PEARL  IMO9060247  ...  27.0    9.8   49.0   
117425  224.1    226.0  OCEAN PEARL  IMO9060247  ...  27.0    9.8   49.0   
117521  223.0    224.0  OCEAN PEARL  IMO9060247  ...  27.0    9.8   49.0   
118718  224.3    225.0  OCEAN PEARL  IMO9060247  ...  27.0    9.8   49.0   
118261  222.3    223.0  OCEAN PEARL  IMO9060247  ...  27.0    9.8   49.0   

        TransceiverClass  LONG_5  LAT_5  LONG_10 LAT_10 LONG_15 LAT_15  
117354       

In [21]:
df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)
print(df_train.head(10))


             id       MMSI        BaseDateTime       LAT       LON   SOG  \
117354  9030354  209513000 2023-05-25 00:00:04  26.34658 -78.94057  31.9   
117425  9035334  209513000 2023-05-25 00:01:06  26.33986 -78.94755  31.9   
117521  9042882  209513000 2023-05-25 00:02:16  26.33224 -78.95551  32.2   
118718  9138497  209513000 2023-05-25 00:11:58  26.26969 -79.02179  31.6   
118261  9101604  209513000 2023-05-25 00:14:42  26.25213 -79.04084  32.3   
119115  9170094  209513000 2023-05-25 00:16:20  26.24117 -79.05147  32.3   
121742  9370175  209513000 2023-05-25 00:18:36  26.22595 -79.06683  32.9   
118618  9129883  209513000 2023-05-25 00:19:42  26.21860 -79.07424  32.5   
118772  9142300  209513000 2023-05-25 00:20:43  26.21183 -79.08091  31.0   
118884  9152712  209513000 2023-05-25 00:21:44  26.20550 -79.08769  32.3   

          COG  Heading   VesselName         IMO  ... Width  Draft  Cargo  \
117354  222.8    224.0  OCEAN PEARL  IMO9060247  ...  27.0    9.8   49.0   
117425  224

C:\Users\Paul\AppData\Local\Temp\ipykernel_26172\703263359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)


In [ ]:
print(f"Taille de l'ensemble d'apprentissage : {len(df_train)}")
print(f"Taille de l'ensemble de test : {len(df_test)}")
print(f"Taille de l'ensemble de validation : {len(df_val)}")


Taille de l'ensemble d'apprentissage : 270763
Taille de l'ensemble de test : 82909
Taille de l'ensemble de validation : 58051


: 